<a href="https://colab.research.google.com/github/LeeGitaek/2020_AI_Class/blob/master/MNIST_Dnn_Xavier_dropout_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl


In [68]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun  5 06:02 kaggle.json


In [48]:
!kaggle competitions download -c 2020aimnist

2020aimnist.zip: Skipping, found more recently modified local copy (use --force to force download)


In [49]:
!unzip 2020aimnist.zip

Archive:  2020aimnist.zip
replace mnist_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mnist_test.csv          
replace mnist_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mnist_train.csv         

replace submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [{ENTER}]
replace submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission.csv          


In [0]:
import pandas as pd
import numpy as np

import torch
import torchvision.datasets as data 
import torchvision.transforms as transforms
import random 

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)
#난수생성 규칙 설정 seed
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
#학습 파라미터 설정
# tpu = 구글에서 만든 gpu

learning_rate = 0.001
training_epochs = 19
batch_size = 100
#drop_prob = 0.3
#모의 문제 해결 수 

In [0]:
train_data=pd.read_csv('mnist_train.csv',header=None,skiprows=1, usecols=range(1,786))
test_data=pd.read_csv('mnist_test.csv',header=None,skiprows=1, usecols=range(1,785))

In [0]:
x_train_data=train_data.loc[:,0:784]
y_train_data=train_data.loc[:,785]

x_train_data=np.array(x_train_data)
y_train_data=np.array(y_train_data)

x_train_data=torch.FloatTensor(x_train_data)
y_train_data=torch.LongTensor(y_train_data)

In [0]:
train_dataset = torch.utils.data.TensorDataset(x_train_data, y_train_data)

In [0]:
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
#shuffle = 데이터 섞어줌
## root 값은 어디에 저장할 것인가 

In [0]:
linear1 = torch.nn.Linear(784,512,bias=True)
linear2 = torch.nn.Linear(512,512,bias=True)
linear3 = torch.nn.Linear(512,512,bias=True)
linear4 = torch.nn.Linear(512,512,bias=True)
linear5 = torch.nn.Linear(512,10,bias=True)
relu = torch.nn.ReLU()
#dropout = torch.nn.Dropout(p=drop_prob)
#레이어를 5개 쌓는다. 
# ( nn + relu ) *5

In [110]:
torch.nn.init.xavier_uniform(linear1.weight)
torch.nn.init.xavier_uniform(linear2.weight)
torch.nn.init.xavier_uniform(linear3.weight)
torch.nn.init.xavier_uniform(linear4.weight)
torch.nn.init.xavier_uniform(linear5.weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  """


Parameter containing:
tensor([[ 0.0532,  0.0559,  0.0808,  ..., -0.1021, -0.0792,  0.0327],
        [-0.0251, -0.0755, -0.0711,  ..., -0.0789, -0.0583,  0.0805],
        [ 0.0589, -0.0162, -0.0925,  ..., -0.0493,  0.0737,  0.0754],
        ...,
        [ 0.0110, -0.0940,  0.0270,  ...,  0.0278, -0.0510,  0.0844],
        [-0.0956,  0.0298, -0.0530,  ...,  0.0773,  0.0433, -0.0057],
        [ 0.0338, -0.0616, -0.0832,  ..., -0.0790,  0.0203, -0.0677]],
       requires_grad=True)

In [0]:
model = torch.nn.Sequential(linear1,relu,
                            linear2,relu,
                            linear3,relu,
                            linear4,relu,
                            linear5).to(device)
#맨 마지막 레이어에서는 relu는 빼준다.

In [0]:
#손실함수와 최적화 함수 
loss = torch.nn.CrossEntropyLoss().to(device) # 소프트맥스 내부적으로 계산 
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate) 

In [113]:
total_batch = len(data_loader)
model.train() # drop_out= true
for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in data_loader:
      # (100,1,28,28) 크기의 텐서를 (100,784) 크기의 텐서로 변형 
       X = X.view(-1,28*28).to(device) #reshape과 유사
       # one - hot 인코딩 되어있지 않음 
       Y = Y.to(device)

       # 그레디언트 초기화
       optimizer.zero_grad()
       hypothesis = model(X)

       cost = loss(hypothesis,Y) 
       cost.backward()
       optimizer.step()

       avg_cost += cost/total_batch
    print('Epoch :','%04d' % (epoch+1),'cost=','{:.9f}'.format(avg_cost))
print('Learning finished..!')      

Epoch : 0001 cost= 0.904932797
Epoch : 0002 cost= 0.139125049
Epoch : 0003 cost= 0.112006970
Epoch : 0004 cost= 0.096241906
Epoch : 0005 cost= 0.085063547
Epoch : 0006 cost= 0.079490796
Epoch : 0007 cost= 0.070525371
Epoch : 0008 cost= 0.063653827
Epoch : 0009 cost= 0.065009356
Epoch : 0010 cost= 0.058195591
Epoch : 0011 cost= 0.054308020
Epoch : 0012 cost= 0.051044174
Epoch : 0013 cost= 0.046213493
Epoch : 0014 cost= 0.043970402
Epoch : 0015 cost= 0.055297222
Epoch : 0016 cost= 0.039156497
Epoch : 0017 cost= 0.035950787
Epoch : 0018 cost= 0.032962471
Epoch : 0019 cost= 0.040366035
Learning finished..!


In [99]:
with torch.no_grad():
    # float() : GPU => CPU 로 데이터 내리기
    #model.eval() # dropout = false 
    x_test_data=test_data.loc[:,:]
    x_test_data=np.array(x_test_data)
    x_test_data=torch.from_numpy(x_test_data).float().to(device)

    prediction = model(x_test_data)
    correct_prediction = torch.argmax(prediction, 1) 
    accuracy = correct_prediction.float().mean()
    print('Accuracy:',accuracy.item())


Accuracy: 4.433099746704102


In [0]:
correct_prediction = correct_prediction.cpu().numpy().reshape(-1,1)

In [101]:
submit=pd.read_csv('submission.csv')
submit

,id,Label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
9995,9995,1
9996,9996,1
9997,9997,1
9998,9998,1


In [102]:
for i in range(len(correct_prediction)):
  submit['Label'][i]=correct_prediction[i].item()

submit

,id,Label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4
...,...,...
9995,9995,2
9996,9996,3
9997,9997,4
9998,9998,5


In [103]:
submit.to_csv('submission_result.csv',index=False,header=True)

!kaggle competitions submit -c 2020aimnist -f submission_result.csv -m "14010974 이기택"

100% 67.3k/67.3k [00:04<00:00, 14.0kB/s]
Successfully submitted to 2020.AI.MNIST